In [2]:
import numpy as np
from numpy.linalg import norm
import cmath
import matplotlib.pyplot as plt

In [178]:
class SparseMatrix:
    
    #rows and columns give the dimensions of the matrix
    def __init__(self,rows,columns):
        
        self.rows = rows
        self.columns = columns
        self.matrix = {}
        
        return 
    
    def setElement(self,i,j,m):
        
   
        self.matrix[(i,j)] = m
    
    
    def getElement(self,i,j):
        
        if (i,j) in self.matrix:
            
            return self.matrix[(i,j)]
        
        else:
            
            return 0
        
    def getHermTranspose(self):
        
        return 
    
    
    def __str__(self):
        
        rep = ''
        
        for i in range(0,self.rows):
            
            row = ''
            for j in range(0,self.columns):
                
                row += str(self.getElement(i,j))+'\t'
                    
            rep += row + '\n'
                
        
        return rep
        
            
    
    # Define inner product
    def innerProduct(self,other):
        
        result = SparseMatrix(self.rows,other.columns)
        
        if self.columns == other.rows:
            
            for (i,j) in self.matrix:
                
                for (k,l) in other.matrix:
                    
                    if (j == k):
                        val = result.getElement(i,l)
                        result.setElement(i, l, val + (self.matrix[(i,j)]*other.matrix[(k,l)]))
            
            
        else:
            
            raise ValueError(
                    'Incompatible Dimentions')


        return result
    
    #Define outer product
    #Other on the right
    def outerProduct(self,other):
            
        result = SparseMatrix(self.rows*other.rows,self.columns*other.columns)
        
        for (i,j) in self.matrix:
            
            for (k,l) in other.matrix:
                
                result.setElement( ((i*other.rows)+k), ((j*other.columns)+l), self.matrix[(i,j)]*other.matrix[(k,l)] )
                
                
        return result
    
    @staticmethod
    def add(m1,m2):
        
        return 
    
    

In [179]:
#Store qubit state

class QuantumRegister(SparseMatrix):
    
    def __init__(self,n_qubits = 1, base_state_ = None):
        
        self.n_states = 2**n_qubits
        self.n_qubits = n_qubits
        
        super(QuantumRegister,self).__init__(self.n_states,1)
        
        
        if base_state_ is None:
            
            self.setElement(0,0,complex(1))
            
        else:
            
            if len(base_state_) == self.n_states:
                for i in range(len(base_state_)):
                
                    if abs(base_state_[i]) != 0:
                        
                        self.setElement(i,0,complex(base_state_[i]))
                
                self.normalize()
                
            else:
            
                raise ValueError(
                    'Length of states is incorrect!'
                    )
            
       
        
    def setState(self,state_):
        
        self.matrix.clear()
        
        if len(state_) == self.n_states:
            
            for i in range(len(state_)):
                
                if abs(state_[i]) != 0:
                    
                    self.setElement(i,0,complex(state_[i]))
                    
            self.normalize()
                    
        else:
            
            raise ValueError(
                'Length of base states is incorrect!')
            
        
    
    def measure(self, doPrint = False):
        
        probabilities_ = np.zeros(self.n_states)
        
        for i in range(self.n_states):
            
            if (i,0) in self.matrix:
                
                probabilities_[i] = abs(self.matrix[(i,0)])**2
                
            else:
            
                probabilities_[i] = 0
    
        state =  int (np.random.choice(self.n_states, p=probabilities_))
        
        if doPrint:
            
            print("The measured state is: |" + np.binary_repr(state, self.n_qubits) +">")
        
        return state
    
    def __str__(self):
        
        rep = ''

        for i in range(self.n_states):
            if (i,0) in self.matrix:

                rep = rep + '({0:+.2f})'.format(self.matrix[(i,0)]) + "*|" + np.binary_repr(i, self.n_qubits) + "> "

            else:
                continue
        return rep
        
    def normalize(self):
        
        norm = 0
        
        for (i,j) in self.matrix:
            
            norm += abs(self.matrix[(i,j)])**2
            
        
        for (i,j) in self.matrix:
            
            self.setElement(i,j,(1/cmath.sqrt(norm))*self.matrix[i,j])

            
    def __mul__ (self,other):
        
        if isinstance(other, QuantumRegister):
            
            result = QuantumRegister(self.n_qubits+other.n_qubits)
            
            result.matrix = self.outerProduct(other).matrix
            
            result.normalize()

            return result
        else:
            raise TypeError('Multiplication not defined between quantum register and {}.'.format(type(other)))
            
            
    def plotRegister(self):
        
        x_ = []
        
        y_ = []
        
        for i in range(self.n_states):
            
            if (i,0) in self.matrix:
                
                x_.append("|"+np.binary_repr(i, self.n_qubits)+">")
                y_.append(abs(self.matrix[(i,0)]))
        
        xpos_ = np.arange(len(x_))
        
        plt.bar(xpos_,y_)
        
        plt.xticks(xpos_, x_)
            
        plt.show()
            
            
    def split(self, n_a, n_b):
        
        if n_a + n_b != self.n_qubits:
            raise ValueError(
                'Number of qubits of subregisters must be '
                'equal to total number of qubits of current register!')

    
        # Calculate number of base states for each subregister
        n = int(2 ** (n_a))
        k = int(2 ** (n_b))
        
        a_states_ = []
        b_states_ = []
        
        for i in range(0, self.n_states, k):
            
            new_b_states_ = [0]*k
            
            for j in range(0,k):
                
                if (i+j,0) in self.matrix:
                
                    new_b_states_[j] = self.matrix[(i+j,0)] 
                else:
                    
                    new_b_states_[j] = complex(0)
             
            
            normal = norm(np.abs(new_b_states_))
            
            if normal != 0:
                new_b_states_ = new_b_states_/normal
            elif normal ==0:
                a_states_.append(0+0.j)
                continue
            
#             print(new_b_states_)
#             print('B states '+ str(b_states_))
            
            if np.array_equal(b_states_,[]):
                
                b_states_ = new_b_states_
                a_states_.append(normal)
                continue
             
            elif not np.array_equal(b_states_,new_b_states_):
#                  Compare absolute values and then if equal, go through each element trying to 
# find a miltiple. if multiple same, append the multiple to a_states_
 

                if np.array_equal(np.abs(b_states_),np.abs(new_b_states_)):
        
                    a_states_holder = new_b_states_[0]/b_states_[0]
            
                    for i in range(1,len(b_states_)):
                    
#                         print('Here'+ '\n'+ str(new_b_states_[i]) +'\n' + str(b_states_[i]))
                        
                        if a_states_holder == new_b_states_[i]/b_states_[i]:
                            
                            continue
                            
                        elif new_b_states_[i] != 0 and b_states_[i] != 0:
                            
                            raise TypeError(
                                'The registers are entangled'
                                    )
                            
                        else:
                            continue
                    a_states_.append(a_states_holder*normal)
                            
        
                else:
                
                    raise TypeError(
                        'The registers are entangled'
                        )
            else:
                    
                a_states_.append(normal)
                continue
                    
         
         
#         print("After done")
#         print(a_states_)
#         print(b_states_)
        a = QuantumRegister(n_a,a_states_)
        b = QuantumRegister(n_b,b_states_)
        
        return(a,b)

In [120]:
reg1 = QuantumRegister(2,[1.j,4,1,0])

reg2 = QuantumRegister(2,[1.j,1.j,0,2])


reg3 = reg1*reg2

print(reg1)


print(reg2)

print(reg3)


reg4 = QuantumRegister(2,[-1,0,0,1])
# a,b = reg4.split(1,1)

# print(a)

# print(b)

(+0.00+0.24j)*|00> (+0.94+0.00j)*|01> (+0.24+0.00j)*|10> 
(+0.00+0.41j)*|00> (+0.00+0.41j)*|01> (+0.82+0.00j)*|11> 
(-0.10+0.00j)*|0000> (-0.10+0.00j)*|0001> (+0.00+0.19j)*|0011> (+0.00+0.38j)*|0100> (+0.00+0.38j)*|0101> (+0.77+0.00j)*|0111> (+0.00+0.10j)*|1000> (+0.00+0.10j)*|1001> (+0.19+0.00j)*|1011> 


These numbers all need to be complex ***** look into how to do this ./
*** Done ***

Create class for sparse matrices

In [193]:
class Operator(SparseMatrix):
    
    def __init__(self, n_qubits : int = 1, base = np.zeros((2,2))):
        
        if n_qubits <= 0 :
            raise ValueError('Operator must operate on at least 1 qubit!')

        self.n_qubits = n_qubits
        self.size = 2 ** n_qubits
        
        act_qubits = int(np.log2(len(base)))
        
        
        base_matrix = SparseMatrix(*[len(base)]*2)
        
        for i in range(0,len(base)):
            
            for j in range(0,len(base)):
                
                if base[i][j] != 0:
                
                    base_matrix.setElement(i,j,complex(base[i][j]))
                    
                else:
                    
                    continue
        
        for i in range(0,n_qubits,act_qubits):
            
            if i == 0:
                
                result = base_matrix
                continue
            
            result = result.outerProduct(base_matrix)
                
            
        super(Operator, self).__init__(self.size,self.size)
        
        self.matrix = result.matrix
            
            
            
    def __mul__(self,rhs):
        
        
        if isinstance(rhs, QuantumRegister):
        
       
            result = QuantumRegister(n_qubits = self.n_qubits)
            
        
        elif isinstance(rhs, Operator):
            
            result = Operator(n_qubits = self.n_qubits)
            
        else :
            " Raise type error if the right type isn't provided"
            raise TypeError(
                'Multiplication not defined for Operator and {}.'.format(type(rhs))
            )
            
        if rhs.n_qubits != self.n_qubits:
                raise ValueError(
                    'Number of states do not correspnd: rhs.n_qubits = {}, lhs.n_qubits = {}'.format(rhs.n_qubits,
                                                                                                     self.n_qubits))

        
        result.matrix = self.innerProduct(rhs).matrix
        
        return result
    

Then write methods to return the specific gates

In [194]:
class Hadamard(Operator):
    """
    Class that defines hadamard gate. This class extends the Operator class.
    """

    def __init__(self, n_qubits: int =1):
        # Define "base" hadamard matrix for one qubit and correponding sparse matrix
        base = 1 / np.sqrt(1) * np.array([[1, 1], [1, -1]])
        super(Hadamard, self).__init__(n_qubits, base)

Define Basic Gates 

0	(1+0j)	
(1+0j)	(-1+0j)	



dict_keys([(1, 2), (2, 1), (3, 2)])


array([[1, 2],
       [3, 4],
       [5, 6]])

(3, 2)

2.0